<a href="https://colab.research.google.com/github/anmyvla-beep/python-ai-Mysttonen-Angelika/blob/main/notebooks/week2b_read_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Week 2: Data Analysis — Чтение и проверка данных о телесериалах

**Цель**: Научиться читать CSV-файлы из репозитория GitHub в Google Colab и выполнять базовую проверку данных о телесериалах с помощью pandas.

**Данные:**
- `tvserialone.csv` — информация о телесериалах: годы выхода, награды и номинации

**Что мы делаем:**
1. Клонируем ваш репозиторий GitHub в Colab
2. Читаем CSV-файл в pandas DataFrame
3. Очищаем и анализируем структуру данных (сериалы, годы, награды)
4. Проверяем качество данных и готовим их к дальнейшему анализу

## 🐱 [1] Клонируем репозиторий курса в Colab

In [1]:
# 🐱 Шаг 1. Клонируем ваш репозиторий в Colab

import os

if not os.path.exists("python-ai-Mysttonen-Angelika"):
    !git clone -q https://github.com/anmyvla-beep/python-ai-Mysttonen-Angelika.git

%cd python-ai-Mysttonen-Angelika

print("✅ Репозиторий готов, теперь мы работаем внутри папки python-ai-Mysttonen-Angelika")

/content/python-ai-Mysttonen-Angelika
✅ Репозиторий готов, теперь мы работаем внутри папки python-ai-Mysttonen-Angelika


## 📥 [2A] Простое чтение CSV-файлов в pandas

Сначала просто прочитаем оба CSV-файла в объекты `DataFrame`, без каких‑либо изменений.

После этого мы узнаем, сколько строк загружено в каждый датасет.

In [12]:
# 🐱 Шаг 2A. Чтение CSV-файлов в pandas

import pandas as pd

df_tv = pd.read_csv("data/tvserialone.csv")

print("✅ Загружено строк в df_cost:", len(df_tv))

✅ Загружено строк в df_cost: 1224


## 🧹 [2B] Очистка и переименование столбцов

В исходном CSV-файле есть **технические столбцы**, которые полезны для Викиданных, но мешают простому анализу:

- Столбец `tv_series` с URL (ссылкой на объект Wikidata) — нам не нужна ссылка, нам достаточно названия сериала.
- Столбцы `tv_seriesLabel`, `awardLabel`, `subjectLabel` содержат читаемые подписи (название сериала, награда, предмет номинации).

В этом шаге мы:
- удалим столбец с URL Wikidata (`tv_series`);
- переименуем `tv_seriesLabel → tv_series`, `awardLabel → award`, `subjectLabel → subject`;
- приведём числовые столбцы (`start_year`, `end_year`) к типу `int`.

При приведении к числам мы используем:

- `pd.to_numeric(..., errors="coerce")` — преобразует значения в числа, некорректные значения превращает в `NaN`;
- `fillna(0)` — заменяет пропущенные значения (`NaN`) на 0;
- `astype(int)` — переводит столбец к целочисленному типу.

> ⚠️ **Важно:** если в ваших данных есть столбцы с URL Wikidata и столбцы вида `*Label`, этот шаг обязателен, чтобы получить аккуратные таблички для анализа.

In [13]:
# 🧹 Шаг 2B. Очистка и переименование столбцов (идемпотентная версия)

# Проверяем, нужно ли выполнять очистку: если есть столбец с суффиксом "Label" — данные ещё не обработаны
if "tv_seriesLabel" in df_tv.columns:
    # 1. Удаляем технический столбец с URL Wikidata (игнорируем ошибку, если столбца уже нет)
    df_tv = df_tv.drop(columns=["tv_series"], errors="ignore")

    # 2. Переименовываем столбцы с читаемыми подписями
    df_tv = df_tv.rename(columns={
        "tv_seriesLabel": "tv_series",
        "awardLabel": "award",
        "subjectLabel": "subject"
    })

    # 3. Приводим числовые столбцы к целочисленному типу
    df_tv["start_year"] = pd.to_numeric(
        df_tv["start_year"], errors="coerce"
    ).fillna(0).astype(int)

    df_tv["end_year"] = pd.to_numeric(
        df_tv["end_year"], errors="coerce"
    ).fillna(0).astype(int)

    print("✅ Данные успешно очищены и подготовлены к анализу")
else:
    print("ℹ️ Данные уже очищены ранее — повторная обработка не требуется")

print("\n📋 Структура датасета:")
display(df_tv.head())
print("\n🔍 Текущие столбцы:", list(df_tv.columns))

✅ Данные успешно очищены и подготовлены к анализу

📋 Структура датасета:


,tv_series,start_year,end_year,award,subject
0,Меня зовут Эрл,2005,2009,Премия «Спутник» за лучший телевизионный сериа...,NaN
1,Меня зовут Эрл,2005,2009,премия «Спутник» за лучшую женскую роль в теле...,NaN
2,Меня зовут Эрл,2005,2009,Премия «Спутник» за лучшую женскую роль второг...,NaN
3,Меня зовут Эрл,2005,2009,Премия Гильдии киноактёров США за лучшую женск...,NaN
4,Меня зовут Эрл,2005,2009,Премия Гильдии режиссёров Америки за лучшую ре...,NaN



🔍 Текущие столбцы: ['tv_series', 'start_year', 'end_year', 'award', 'subject']


## 🔍 [3] Обзор данных: структура и первые строки

Сделаем короткий обзор нашего DataFrame с телесериалами:

- посмотрим размер таблицы (`shape`);
- выведем список столбцов;
- посмотрим первые несколько строк для понимания структуры;
- дополнительно проанализируем распределение по годам выхода сериалов и количеству наград.

Для удобства напишем функцию `show_info(df, name)`, которая выводит краткую сводку по любому датасету.

In [15]:
def show_info(df, name, n=10):
    """Краткий обзор DataFrame: имя, размер, список столбцов и первые строки."""
    print(f"\n📊 {name}")
    print("Размер:", df.shape)
    print("Столбцы:", ", ".join(df.columns))
    print("\nПервые строки:")
    display(df.head(n))

# 🔍 Шаг 3. Обзор данных

show_info(df_tv, "Телесериалы и их награды (df_tv)")

print("\n📈 Статистика по годам выхода:")
print(f"  Первый сериал в датасете: {df_tv['start_year'].min()} год")
print(f"  Последний сериал в датасете: {df_tv['end_year'].max()} год")
print(f"  Средняя продолжительность сериала: {(df_tv['end_year'] - df_tv['start_year']).mean():.1f} лет")

print("\n🏆 Количество уникальных записей:")
print(f"  Уникальных сериалов: {df_tv['tv_series'].nunique()}")
print(f"  Уникальных наград: {df_tv['award'].nunique()}")
print(f"  Уникальных номинаций/тем: {df_tv['subject'].nunique()}")


📊 Телесериалы и их награды (df_tv)
Размер: (1224, 5)
Столбцы: tv_series, start_year, end_year, award, subject

Первые строки:


,tv_series,start_year,end_year,award,subject
0,Меня зовут Эрл,2005,2009,Премия «Спутник» за лучший телевизионный сериа...,NaN
1,Меня зовут Эрл,2005,2009,премия «Спутник» за лучшую женскую роль в теле...,NaN
2,Меня зовут Эрл,2005,2009,Премия «Спутник» за лучшую женскую роль второг...,NaN
3,Меня зовут Эрл,2005,2009,Премия Гильдии киноактёров США за лучшую женск...,NaN
4,Меня зовут Эрл,2005,2009,Премия Гильдии режиссёров Америки за лучшую ре...,NaN
5,Меня зовут Эрл,2005,2009,прайм-тайм премия «Эмми» в категории «Лучший п...,NaN
6,Меня зовут Эрл,2005,2009,Премия Ассоциации телевизионных критиков за вы...,NaN
7,Меня зовут Эрл,2005,2009,Премия «Золотой глобус» за лучший телевизионны...,NaN
8,Меня зовут Эрл,2005,2009,Primetime Emmy Award for Outstanding Casting f...,NaN
9,Меня зовут Эрл,2005,2009,Primetime Emmy Award for Outstanding Cinematog...,NaN



📈 Статистика по годам выхода:
  Первый сериал в датасете: 0 год
  Последний сериал в датасете: 2024 год
  Средняя продолжительность сериала: -202.7 лет

🏆 Количество уникальных записей:
  Уникальных сериалов: 294
  Уникальных наград: 277
  Уникальных номинаций/тем: 73


## 🔬 [4] Глубокая аналитика: эпохи признания и темы номинаций

Мы проведём два уникальных анализа, раскрывающих **культурную историю телевидения** через призму наград:

## 🔬 [4] Глубокая проверка данных: эпохи признания и темы номинаций

Наши данные раскрывают не просто список сериалов — они отражают **культурные приоритеты телевизионной индустрии** через призму наград. Проведём два ключевых анализа:

### 📅 Эпохи признания
Разобьём сериалы по десятилетиям старта (`start_year`) и посчитаем:
- сколько наград было вручено сериалам каждой эпохи;
- как менялось «премиальное разнообразие» (уникальные награды на десятилетие).

Это покажет, в какие периоды телевидение активнее отмечало достижения — возможно, отражая золотые эры жанров или появление новых премий.

### 💬 Темы номинаций: что ценит индустрия?
Столбец `subject` содержит темы/категории номинаций: «семья», «преступность», «полиция» и др. Проанализируем:
- **Топ-15 самых частых тем** — что чаще всего отмечают наградами;
- **Редкие темы** — уникальные номинации, вручённые лишь одному сериалу.

Это раскроет культурные коды: какие сюжетные линии и социальные темы индустрия считает достойными признания.

### 🤝 Особый фокус: дружба на экране
В конце выведем все сериалы, где в номинации упоминается **дружба** — чтобы увидеть, как телевидение отражает эту универсальную ценность.

In [17]:
# 🔬 Шаг 4. Глубокая проверка: эпохи признания и темы номинаций

import pandas as pd

# Фильтруем корректные данные (игнорируем строки с годом = 0 или будущими датами)
df_valid = df_tv[df_tv["start_year"].between(1900, 2024)].copy()

# === АНАЛИЗ 1: ЭПОХИ ПРИЗНАНИЯ ===
print("📅 ЭПОХИ ПРИЗНАНИЯ: награды по десятилетиям старта сериалов")
print("=" * 70)

# Определяем десятилетие для каждого сериала
df_valid["decade"] = (df_valid["start_year"] // 10) * 10

# Группируем по десятилетию
decade_stats = df_valid.groupby("decade").agg(
    total_nominations=("award", "count"),
    unique_series=("tv_series", "nunique"),
    unique_awards=("award", "nunique")
).sort_index()

# Считаем "премиальное разнообразие" — уникальных наград на сериал
decade_stats["awards_per_series"] = (
    decade_stats["unique_awards"] / decade_stats["unique_series"]
).round(2)

print(decade_stats.to_string())
print(f"\n💡 Всего учтено сериалов с корректными годами: {len(df_valid)} из {len(df_tv)}")

# === АНАЛИЗ 2: ТЕМЫ НОМИНАЦИЙ ===
print("\n\n💬 ТЕМЫ НОМИНАЦИЙ: что ценит телевизионная индустрия?")
print("=" * 70)

# Убираем пустые значения в темах
subjects_clean = df_tv["subject"].dropna().str.strip()
subjects_clean = subjects_clean[subjects_clean != ""]

print(f"\nВсего номинаций с указанной темой: {len(subjects_clean)}")
print(f"Уникальных тем: {subjects_clean.nunique()}")

print("\n🏆 Топ-15 самых частых тем номинаций:")
top_subjects = subjects_clean.value_counts().head(15)
for rank, (subject, count) in enumerate(top_subjects.items(), 1):
    pct = count / len(subjects_clean) * 100
    print(f"{rank:2d}. {subject:.<40} {count:3d} ({pct:4.1f}%)")

print("\n💎 Редкие темы (вручены только одному сериалу):")
rare_subjects = subjects_clean.value_counts()
rare_count = (rare_subjects == 1).sum()
print(f"   Таких тем: {rare_count} из {subjects_clean.nunique()}")
print(f"   Примеры: {', '.join(rare_subjects[rare_subjects == 1].index[:5])}")

# === СПЕЦИАЛЬНЫЙ ЗАПРОС: СЕРИАЛЫ ПРО ДРУЖБУ ===
print("\n\n🤝 СЕРИАЛЫ С НОМИНАЦИЯМИ, СВЯЗАННЫМИ С ДРУЖБОЙ")
print("=" * 70)

# Регистронезависимый поиск "дружба" в темах
friendship_mask = df_tv["subject"].str.contains("дружба", case=False, na=False)
df_friendship = df_tv[friendship_mask][["tv_series", "start_year", "end_year", "award", "subject"]].drop_duplicates()

if len(df_friendship) > 0:
    print(f"Найдено {len(df_friendship)} уникальных номинаций со словом «дружба»:\n")
    for idx, row in df_friendship.iterrows():
        years = f"{row['start_year']}-{row['end_year']}" if row['end_year'] > 0 else str(row['start_year'])
        print(f"📺 {row['tv_series']} ({years})")
        print(f"   Награда: {row['award']}")
        print(f"   Тема:    {row['subject']}\n")
else:
    print("❌ В датасете не найдено номинаций, содержащих слово «дружба».")
    print("\n🔎 Возможные причины:")
    print("   • Темы номинаций о дружбе описываются другими словами («отношения», «команда»)")
    print("   • Данные из Викиданных неполны для этой категории")
    print("   • Поиск чувствителен к языку: проверьте английские варианты («friendship»)")

📅 ЭПОХИ ПРИЗНАНИЯ: награды по десятилетиям старта сериалов
        total_nominations  unique_series  unique_awards  awards_per_series
decade                                                                    
1930                   12              1             12              12.00
1950                   16              5             16               3.20
1960                   36              9             31               3.44
1970                   62             11             48               4.36
1980                   45             13             42               3.23
1990                  337             48            110               2.29
2000                  432             68            134               1.97
2010                  177             70            113               1.61
2020                   76             43             31               0.72

💡 Всего учтено сериалов с корректными годами: 1193 из 1224


💬 ТЕМЫ НОМИНАЦИЙ: что ценит телевизионная индустрия?



## 📝 Summary

**Что мы сделали в этом ноутбуке (Week 2):**

- ✅ Клонировали репозиторий GitHub в Colab
- ✅ Прочитали 2 CSV-файла из `data/examples/`
- ✅ Удалили URL Wikidata и переименовали столбцы (`*Label → короткие имена`)
- ✅ Проверили структуру данных (размер, столбцы, первые строки)
- ✅ Посмотрели базовую статистику по бюджету (`capital_cost`)
- ✅ Выполнили быструю валидацию:
  - количество уникальных фильмов, стран, жанров
  - диапазоны значений
  - топ стран и жанров по числу записей

Теперь у нас есть **аккуратные, проверенные таблицы**, с которыми удобно работать дальше.

В отдельном ноутбуке для следующей недели мы будем использовать **те же данные** для:
- более сложного анализа (группировки, фильтрация),
- и построения визуализаций (графики и диаграммы). 🎨